In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)

os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.metrics import  roc_auc_score

import pickle 
import logging


# from config.config import SQLQuery
# querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code


In [2]:
from preprocess import Convert,MissingValues,Outlier,FeatureSelection
from feature_transformation import Scaler,Transform,Selection
from model_building import split_test_train, feature_encoding, classification_models
from model_evaluations import model_metrics, feature_importance, probability_bins, cross_validation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

from statsmodels.stats.outliers_influence import variance_inflation_factor

# object initiation 
tf = Transform()
sel = Selection()
ft = FeatureSelection()
cv = Convert()
mv = MissingValues()
ot = Outlier()

# set seed
seed = 37

In [12]:
df_raw = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/final_dataset_2.pkl')
df_raw.shape

(646, 255)

In [13]:
feat_list = ['credit_size_3M',
 'pos_size_1M',
 'ratio_amt_credits_less_100_credits_grt_500_3M',
 'median_credit_amt_1M',
 'ratio_pos_size_1M_6M',
 'median_pos_amt_3M',
 'count_credits_grt_500_3M',
 'loans_size_6M',
 'ach_count_6M']

In [14]:
keep_cols = ['business_id', 'lending_business_id', 'decision_date', 'target', 'txn_flag']

In [15]:
df_raw = df_raw[keep_cols + feat_list]
df_raw.shape

(646, 14)

In [16]:
df_raw.sample(5)

,business_id,lending_business_id,decision_date,target,txn_flag,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
613,b6398e52-5930-4d55-9b0c-d067e9d4ebba,c4f03bd8-cf2a-4b99-9182-f9ab55ba3f9e,2023-01-11,0,1.0,412.105333,NaN,0.064512,97.500,NaN,NaN,5.0,NaN,1.0
532,325c2041-db27-4013-959e-962d852726f2,48e377ad-8569-4a4b-8f4f-cdad1d57fafc,2023-01-24,1,1.0,391.122414,NaN,0.046342,315.000,NaN,NaN,10.0,92.857143,1.0
395,42a374fb-8345-461d-9903-1a11d6b91f58,ce356df6-6386-4070-b310-239fe02d8a22,2023-02-19,0,1.0,191.785000,NaN,0.161311,40.000,NaN,850.0,4.0,NaN,NaN
127,7307d6c7-73c9-49fb-8c04-3f65cf32a77f,6eadb488-61da-405e-9943-ba5efd33b619,2023-01-30,1,1.0,555.109744,91.484615,0.041819,100.000,0.806718,100.0,6.0,NaN,NaN
96,9a4dbc35-c5c9-49be-bdbd-44ab152158d6,8f32cf89-7449-4ff9-8eeb-281dff282591,2023-01-12,0,1.0,803.445882,NaN,0.008582,447.375,NaN,NaN,9.0,NaN,4.0


In [17]:
df_raw.describe()

,txn_flag,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
count,646.000000,620.000000,256.000000,4.870000e+02,573.000000,256.000000,363.000000,579.000000,129.000000,203.000000
mean,0.986068,1900.288971,1003.794158,5.192965e-02,1189.955541,0.743371,1188.203140,14.096718,3947.993256,12.753695
std,0.117299,4530.977238,3305.248697,1.634456e-01,3382.613029,0.576655,3383.187711,15.195985,7238.932550,25.483802
min,0.000000,0.155000,0.150000,7.307604e-08,0.010000,0.001247,0.010000,1.000000,0.290000,1.000000
25%,1.000000,390.699894,89.477500,2.118917e-03,118.860000,0.331781,100.000000,5.000000,150.000000,2.000000
50%,1.000000,774.588458,281.750000,9.341087e-03,369.600000,0.692535,300.000000,10.000000,783.333333,5.000000
75%,1.000000,1663.454220,920.205937,3.279657e-02,1000.000000,0.998628,882.130000,17.000000,4423.255556,14.500000
max,1.000000,59653.400000,46250.000000,1.794795e+00,51080.000000,4.346746,40480.000000,137.000000,50670.975000,263.000000


In [18]:
median_dict = {'credit_size_3M' : 774.588458,
                'pos_size_1M' : 281.750000,
                'ratio_amt_credits_less_100_credits_grt_500_3M' : 9.341087e-03,
                'median_credit_amt_1M' : 369.600000,
                'ratio_pos_size_1M_6M' : 0.692535,
                'median_pos_amt_3M' : 300,
                'count_credits_grt_500_3M' : 10,
                'loans_size_6M' : 783.3,
                'ach_count_6M' : 5}

In [21]:
df_raw[df_raw['txn_flag'] == 0] = df_raw[df_raw['txn_flag'] == 0].fillna(median_dict)

In [26]:
df_raw = df_raw.fillna(0)
df_raw.isnull().sum().sum()

0

In [ ]:
df_raw.drop('txn_flag',axis=1,inplace=True)

In [27]:
feat_list

['credit_size_3M',
 'pos_size_1M',
 'ratio_amt_credits_less_100_credits_grt_500_3M',
 'median_credit_amt_1M',
 'ratio_pos_size_1M_6M',
 'median_pos_amt_3M',
 'count_credits_grt_500_3M',
 'loans_size_6M',
 'ach_count_6M']

In [31]:
transformed_vars = df_raw.copy()

In [32]:
# 1
# transform
col         = 'credit_size_3M'
conditions  = [ transformed_vars[col] < 178.8,
                (transformed_vars[col] >= 178.9 ) & (transformed_vars[col] < 392.56),
                (transformed_vars[col] >= 392.56 ) & (transformed_vars[col] < 2274.8),
                transformed_vars[col] >= 2274.8 ]

choices     = [-1.056997,-0.484478,0.184337,1.294378]
    
transformed_vars["credit_size_3M"] = np.select(conditions, choices, default=np.nan)


# 2
# transform
col         = 'pos_size_1M'
conditions  = [ transformed_vars[col] < 235.11, 
                (transformed_vars[col] >= 235.11 ) & (transformed_vars[col] < 979.44), 
                transformed_vars[col] >= 979.44 ]

choices     = [-0.152312,0.159398,1.699843]
    
transformed_vars["pos_size_1M"] = np.select(conditions, choices, default=np.nan)

# 3
# transform
col         = 'ratio_amt_credits_less_100_credits_grt_500_3M'
conditions  = [ transformed_vars[col] <= 0,
                transformed_vars[col] > 0 ]

choices     = [-0.112536,0.045514]
    
transformed_vars["ratio_amt_credits_less_100_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

# 4
# transform
col         = 'median_credit_amt_1M'
conditions  = [ transformed_vars[col] < 102,
                (transformed_vars[col] >= 102 ) & (transformed_vars[col] < 475.2),
                transformed_vars[col] >= 475.2 ]

choices     = [-0.444738,-0.119068,0.550264]
    
transformed_vars["median_credit_amt_1M"] = np.select(conditions, choices, default=np.nan)


# 5
# transform
col         = 'ratio_pos_size_1M_6M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 1.21),
                transformed_vars[col] >= 1.21 ]

choices     = [-0.164722,0.141314,1.140227]
    
transformed_vars["ratio_pos_size_1M_6M"] = np.select(conditions, choices, default=np.nan)

# 6
# transform
col         = 'median_pos_amt_3M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 486.25),
                transformed_vars[col] >= 486.25 ]

choices     = [-0.25334,-0.127284,0.968955]
    
transformed_vars["median_pos_amt_3M"] = np.select(conditions, choices, default=np.nan)

# 7
# transform
col         = 'count_credits_grt_500_3M'
conditions  = [ transformed_vars[col] < 1.5,
                (transformed_vars[col] >= 1.5 ) & (transformed_vars[col] < 15.5),
                transformed_vars[col] >= 15.5 ]

choices     = [-0.450862,-0.000945,0.284561]
    
transformed_vars["count_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

# 8
# transform
col         = 'loans_size_6M'
conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 3317),
                transformed_vars[col] >= 3317 ]

choices     = [0.08126,-0.605012,0.814805]
    
transformed_vars["loans_size_6M"] = np.select(conditions, choices, default=np.nan)


# 9
# transform
col         = 'ach_count_6M'
conditions  = [ transformed_vars[col] < 6.5,
                transformed_vars[col] >= 6.5 ]

choices     = [-0.022101,0.149246]
    
transformed_vars["ach_count_6M"] = np.select(conditions, choices, default=np.nan)


In [34]:
transformed_vars.head()

,business_id,lending_business_id,decision_date,target,txn_flag,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,0,1.0,0.184337,-0.152312,0.045514,-0.119068,-0.164722,-0.127284,-0.000945,0.081260,-0.022101
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,0,1.0,0.184337,-0.152312,0.045514,0.550264,-0.164722,-0.127284,0.284561,-0.605012,-0.022101
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,0,1.0,1.294378,-0.152312,-0.112536,-0.444738,-0.164722,-0.253340,-0.450862,0.081260,-0.022101
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,0,1.0,1.294378,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,0,1.0,0.184337,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101


In [37]:
logreg_model = pickle.load(open('/Users/shashankgupta/Documents/code/git_project/plaid_credit/models/final_model.pkl', 'rb'))

In [42]:
# Scoring on full-data

scored_df = transformed_vars.copy()
scored_df['pred_proba'] = logreg_model.predict_proba(scored_df[feat_list])[:,1]

scored_df['odds'] = scored_df['pred_proba'] / (1-scored_df['pred_proba'])
scored_df['log_odds'] = np.log(scored_df['odds'])
scored_df['Ext_IN1_score'] = 601.7 - (86.6 * scored_df['log_odds'])
scored_df.Ext_IN1_score.describe()

count    646.000000
mean     618.060330
std       76.272747
min      469.821422
25%      567.695876
50%      601.579053
75%      655.069967
max      887.743320
Name: Ext_IN1_score, dtype: float64

In [43]:
scored_df.head()

,business_id,lending_business_id,decision_date,target,txn_flag,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M,pred_proba,odds,log_odds,Ext_IN1_score
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,0,1.0,0.184337,-0.152312,0.045514,-0.119068,-0.164722,-0.127284,-0.000945,0.081260,-0.022101,0.521805,1.091197,0.087275,594.141987
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,0,1.0,0.184337,-0.152312,0.045514,0.550264,-0.164722,-0.127284,0.284561,-0.605012,-0.022101,0.596929,1.480956,0.392687,567.693263
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,0,1.0,1.294378,-0.152312,-0.112536,-0.444738,-0.164722,-0.253340,-0.450862,0.081260,-0.022101,0.231759,0.301675,-1.198407,705.482011
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,0,1.0,1.294378,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101,0.240053,0.315882,-1.152387,701.496684
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,0,1.0,0.184337,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101,0.439750,0.784917,-0.242177,622.672553


In [44]:
def bin_logic(x):
    if x < 0.261540:
        return 5
    if x < 0.455167:
        return 4
    if x < 0.530817:
        return 3
    if x < 0.628923:
        return 2
    return 1

In [45]:
scored_df['Ext_IN1_bin'] = scored_df['pred_proba'].apply(lambda x: bin_logic(x))

In [46]:
scored_df.head()

,business_id,lending_business_id,decision_date,target,txn_flag,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M,pred_proba,odds,log_odds,Ext_IN1_score,Ext_IN1_bin
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,0,1.0,0.184337,-0.152312,0.045514,-0.119068,-0.164722,-0.127284,-0.000945,0.081260,-0.022101,0.521805,1.091197,0.087275,594.141987,3
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,0,1.0,0.184337,-0.152312,0.045514,0.550264,-0.164722,-0.127284,0.284561,-0.605012,-0.022101,0.596929,1.480956,0.392687,567.693263,2
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,0,1.0,1.294378,-0.152312,-0.112536,-0.444738,-0.164722,-0.253340,-0.450862,0.081260,-0.022101,0.231759,0.301675,-1.198407,705.482011,5
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,0,1.0,1.294378,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101,0.240053,0.315882,-1.152387,701.496684,5
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,0,1.0,0.184337,-0.152312,-0.112536,0.550264,-0.164722,-0.253340,-0.000945,0.081260,-0.022101,0.439750,0.784917,-0.242177,622.672553,4


In [48]:
scored_df[scored_df['txn_flag']==0]

,business_id,lending_business_id,decision_date,target,txn_flag,credit_size_3M,pos_size_1M,ratio_amt_credits_less_100_credits_grt_500_3M,median_credit_amt_1M,ratio_pos_size_1M_6M,median_pos_amt_3M,count_credits_grt_500_3M,loans_size_6M,ach_count_6M,pred_proba,odds,log_odds,Ext_IN1_score,Ext_IN1_bin
38,1ae5e987-6009-490e-98ff-c1fc3be4cbb6,440f82f2-2c16-4905-9580-6ec1e5e6d29d,2023-03-15,0,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3
46,5a740c4c-94f1-48c9-b7a2-5b1747eb870f,11e26497-4c56-48b6-9b2b-4181f23d6320,2023-03-02,0,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3
47,c037cf47-e730-463b-af87-9c5f4746972b,676a3631-32b4-4880-8265-f64028cbb0d3,2023-01-30,0,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3
211,5c1142e4-4a88-4f46-a0e1-2809f55ab9aa,0549c8e5-2e1c-491b-93d1-1caa8a078c38,2023-03-28,0,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3
421,e62ca860-e7a0-4779-8815-3bfc7e45bc47,b854768e-1903-41e9-ad66-aae065da2773,2023-05-16,0,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3
424,d47fc09c-10ab-4276-9b66-3957876a854b,44e6bb2b-7aae-425f-a734-d96ba6965368,2023-02-23,0,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3
427,8ab5917f-7c14-4f50-abaf-985c4615a743,d6b3c154-d004-4a49-a00a-a9474fa2538f,2023-01-10,1,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3
483,ae44b23b-f848-4907-bdd9-da65200dcb42,84a1f68d-4193-4f87-8048-828f62b2c45a,2023-02-21,0,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3
539,51661ae1-9374-4fcf-9985-990292378faf,4d5a26e7-97a6-41d7-991a-06df1475de65,2023-04-18,0,0.0,0.184337,0.159398,0.045514,-0.119068,0.141314,-0.127284,-0.000945,-0.605012,-0.022101,0.500349,1.001398,0.001397,601.579053,3


In [49]:
# scored_df.to_csv('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/scored_dataset.csv')

In [47]:
# scored_df.to_pickle('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/scored_dataset.pkl')